# Scikit-learn + Dask

Scikit-learn is a popular machine learning library, and implements the minimal API required for training machine learning models. Their library encompasses a wide variety of problem formulations, ranging from simple linear models to more complex clustering or decomposition formulations. Scikit-learn is hindered by computation (what machine library isn't?). This has even gotten a page in their user guide, "[Strategies to scale computationally: bigger data][strat]".

Using scikit-learn estimators with Dask-ML helps scale to large datasets. All the computation remains on scikit-learn's shoulders but the data management is handled by Dask. This allows scaling to large datasets distributed across many machines, or to datasets that do not fit in memory.

This example wraps a scikit-learn estimator with a Dask-ML estimator suited for incremental learning. This Dask-ML estimator manages feeding data to Scikit-learn, and leaves the computation to Scikit-learn. This example will not cover the other algorithms inside of Dask-ML that are well-suited for Dask's distributed architecture.

This example will show

* wrapping a scikit-learn estimator that implement `partial_fit` with `dask_ml`
* training, predicting, and scoring on this wrapped classifier
* integration with other parts of sklearn (e.g., with GridSearchCV)

<img src="http://scikit-learn.org/stable/_static/scikit-learn-logo-small.png">
<img src="https://www.continuum.io/sites/default/files/dask_stacked.png" width="100px">

[strat]:http://scikit-learn.org/stable/modules/scaling_strategies.html


## Data creation
We will create many synthetic data. We are more interested in showing the plumbing this data can flow through than cool insights we can glean from the data.

Let's create some synthetic data that's not too large (so it runs in a reasonable time for you) and but is realistically large. We have 100,000 examples and 100 features in this dataset we create.

In [ ]:
import sklearn.datasets
import dask.array as da
import numpy as np
n, d = int(100e3), 100

X = da.random.normal(size=(n, d), chunks=n // 10)
w_star = da.exp(da.random.uniform(size=d, chunks=d))
w_star = w_star**4
noise = da.random.normal(size=n, chunks=n) * d / 1
y = da.sign(X @ w_star + noise)

We are interested in evaluating this model, so we don't want to learn from test data (ever hear of teaching to the test?). This prevents against that.

In [ ]:
from dask_ml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)

## Model creation

Let's create an sklearn model as usual:

In [ ]:
from sklearn.linear_model import SGDClassifier

est = SGDClassifier(loss='log', penalty='l1', tol=1e-3)

Now, let's wrap it with `dask_ml.wrappers.Incremental`. This will allow this model to scale well to larger datasets that Dask holds.

In [ ]:
from dask_ml.wrappers import Incremental

inc = Incremental(est, scoring='accuracy')

Our model (SGDClassifier here) must implement `partial_fit` to be used with `Incremental`. A list of models that implement this API is available on a scikit-learn documentation page under "Incremental learning": http://scikit-learn.org/stable/modules/scaling_strategies.html#incremental-learning

`Incremental` does data management: it calls `est.partial_fit` on each chunk of the passed data. Dask moves the model to different chunks of the data (or vice versa) to let this happen. Note: Dask-ML `Incremental` gives data to Scikit-learn and does not change the underlying optimization algorithm that Scikit-learn uses. There are different optimization algorithms in Dask-ML that do this, especially for clustering and matrix decomposition.

It is important to specify the scoring parameter in `Incremental`; otherwise, scikit-learn scorers are fed Dask arrays (which they're not optimized for).


[1]:http://scikit-learn.org/stable/modules/scaling_strategies.html
[inc]:http://scikit-learn.org/stable/modules/scaling_strategies.html#incremental-learning

## Model training
For an `Incremental` model, `partial_fit` and `fit` aliased to the same item and do one complete pass over the dataset. Let's call `partial_fit` a couple times, and score it after it's called each time (which will allow for a visualization):

In [ ]:
%%time
# takes about 1min

from distributed.metrics import time
data = []
start = time()
for iteration in range(10):
    inc.partial_fit(X_train, y_train, classes=da.unique(y))
    data += [{'score': inc.score(X_test, y_test),
             'epochs': iteration + 1,
             'time': time() - start}]
    print(data[-1])

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.DataFrame(data)
df.plot(x='epochs', y='score')
plt.title('Score over iterations')
plt.ylabel('Score')
plt.show()

## Predicting
Let's predict, than compute the accuracy

In [ ]:
y_hat = inc.predict(X_test)
errors = np.abs(y_hat - y_test) / 2   # divide by 2 to get 0-1 labels
accuracy = 1 - errors.mean()
accuracy.compute()

In [ ]:
inc.score(X_test, y_test)

## Grid search

This accuracy might be the best possible, but there a lot of parameters to feed `SGDClassifier` that can drastically change the performance. We'll consider two of the most basic parameters, specifying a different loss function and different amount of penalty. This is the problem of hyperparameter optimization, something Dask-ML supports: https://dask-ml.readthedocs.io/en/latest/hyper-parameter-search.html

One algorithm that Dask-ML implements a grid search algorithm through `GridSearchCV`. This algorithm takes in a list of values to evaluate and score each model at every possible combination on values.

In [ ]:
from dask_ml.model_selection import GridSearchCV
params = {'alpha': np.logspace(-6, 0, num=10),
          'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron']}
grid = GridSearchCV(est, params, return_train_score=False)

Notice that we pass the scikit-learn estimator here. `Incremental` is only a small wrapper for data management, and `GridSearchCV` is also a dask tool that for data management.

In [ ]:
%%time
# took about 3 minutes with 8 workers
grid.fit(X, y)

In [ ]:
grid.best_params_

For the best loss, let's see how the score changed over different regularization constants `alpha`:

In [ ]:
df = pd.DataFrame(grid.cv_results_)
show = df[df.param_loss == grid.best_params_['loss']]
show.plot(x='param_alpha', y='mean_test_score', yerr='std_test_score',
          logx=True)

But what if other losses were equally good? Let's see a visualization of the different losses *and* different regularization parameters.

In [ ]:
import seaborn as sns

show = df.pivot_table(index='param_loss', columns='param_alpha', values='mean_test_score')
show.columns = ['%0.3f' % np.log10(x) for x in show.columns]
show.columns.name = 'log10(param_alpha)'

sns.heatmap(show, fmt='0.2f', cmap='magma', cbar_kws={'label': 'score'})
_ = plt.xticks(rotation=45)